<center><img src="../img/logo.png" alt="logo empresa" height="200"> </center>
<center><h1>Project:“AUTO MOTORS”</h1></center>
<center><p> Analyze the impact of the pandemic in this company</p></center>

In [2]:
import pandas as pd 
import seaborn as sns
ventas = pd.read_csv("../data/ventas_updated.csv")

In [3]:
ventas.head()

,Unnamed: 0,IdCliente,NombreCliente,Fecha,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
0,0,C1070956433,PINTO JOHN,2016-01-09,Tello German,LL-2R-0038,PILOT STREET 110/80-17,101.0,LLANTA,1,27.228146,Puente aranda,Calle 13,35.0
1,1,C1032372566,CESAR CRUZ,2016-01-09,Javier Garay,LL-TCA-0133,TOURING 175/70R13,101.0,LLANTA,2,38.119404,Suba,Suba,32.0
2,2,C2865810,PINEDA JAIME,2016-01-09,Marcos Luna,LL-TCA-0019,ENERGY XM2 185/65R14,101.0,LLANTA,1,40.659027,Usaquen,Santa ana,33.0
3,3,C11210285,VILLALBA RODRIGUEZ FABIO,2016-01-09,Tello German,LL-TCA-0009,ENERGY XM2 165/65R13,101.0,LLANTA,2,55.182530,Puente aranda,Calle 13,35.0
4,4,C80755314,MOLINA RONALD,2016-01-09,Javier Garay,LL-TCA-0132,NO UTILIZAR,101.0,LLANTA,4,76.239018,Suba,Suba,32.0


In [4]:
#eliminando Unnamed:0
ventas.drop("Unnamed: 0", axis=1, inplace=True)

<center><h1>4. MAPS(EDA)</h1></center>


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from geopy.geocoders import Nominatim

In [6]:
#Conociendo el volumen de localidades
ventas.Localidad.value_counts()

Engativa          59170
Suba              16653
Barrios unidos    11996
Puente aranda      9904
Usaquen            8308
Bogota             2763
Name: Localidad, dtype: int64

In [7]:
#Conociendo la variedad de Sedes
ventas.Sede.value_counts()

Calle 80           48753
Suba               16653
Av Chile           11996
Ventas externas    10417
Calle 13            9904
Santa ana           8308
Grandes flotas      2763
Name: Sede, dtype: int64

Añadiendo lat and log al df como columnas df[“latitude”], df[“longitude”] en base a la columna Localidad

In [8]:
#Para asegurarnos que es Colombia lo que va a leer y evitar la coincidencia con otros puntos del mundo que se llamen igual, 
#Cambiamos los nombres de Localidad a su nombre en Colombia :
ventas['Localidad'].replace({'Barrios unidos': 'Av Chile, Barrios unidos, Bogotá', 
                                    'Puente aranda': 'Calle 13, Puente Aranda, Bogotá',
                                    'Engativa': 'Calle 80, Engativa, Bogotá',
                                    'Bogota': 'Grandes flotas, Bogota', 
                                    'Usaquen': 'Santa ana, Usaquen, Bogotá',
                                    'Suba': 'Suba, Suba, Bogotá',
                                      }, inplace=True)

In [9]:
#creando una lista con los nombres de las Localidades
localidades = ventas["Localidad"].unique()
localidades

array(['Calle 13, Puente Aranda, Bogotá', 'Suba, Suba, Bogotá',
       'Santa ana, Usaquen, Bogotá', 'Calle 80, Engativa, Bogotá',
       'Av Chile, Barrios unidos, Bogotá', 'Grandes flotas, Bogota'],
      dtype=object)

In [10]:
ventas.head()

,IdCliente,NombreCliente,Fecha,Empleado,Referencia,Descripcion,CodigoFamilia,Familia,Cantidad,Ventas,Localidad,Sede,Area
0,C1070956433,PINTO JOHN,2016-01-09,Tello German,LL-2R-0038,PILOT STREET 110/80-17,101.0,LLANTA,1,27.228146,"Calle 13, Puente Aranda, Bogotá",Calle 13,35.0
1,C1032372566,CESAR CRUZ,2016-01-09,Javier Garay,LL-TCA-0133,TOURING 175/70R13,101.0,LLANTA,2,38.119404,"Suba, Suba, Bogotá",Suba,32.0
2,C2865810,PINEDA JAIME,2016-01-09,Marcos Luna,LL-TCA-0019,ENERGY XM2 185/65R14,101.0,LLANTA,1,40.659027,"Santa ana, Usaquen, Bogotá",Santa ana,33.0
3,C11210285,VILLALBA RODRIGUEZ FABIO,2016-01-09,Tello German,LL-TCA-0009,ENERGY XM2 165/65R13,101.0,LLANTA,2,55.182530,"Calle 13, Puente Aranda, Bogotá",Calle 13,35.0
4,C80755314,MOLINA RONALD,2016-01-09,Javier Garay,LL-TCA-0132,NO UTILIZAR,101.0,LLANTA,4,76.239018,"Suba, Suba, Bogotá",Suba,32.0


In [11]:
#creando un dataframe de Localidades
df_localidades = pd.DataFrame(localidades, columns=["direccion"])

In [12]:
df_localidades

,direccion
0,"Calle 13, Puente Aranda, Bogotá"
1,"Suba, Suba, Bogotá"
2,"Santa ana, Usaquen, Bogotá"
3,"Calle 80, Engativa, Bogotá"
4,"Av Chile, Barrios unidos, Bogotá"
5,"Grandes flotas, Bogota"


In [13]:
#Observamos un problema en Grandes flotas, Bogotá, que no está definido, vamos a reasignarlo con "Grandes flotas, Bogotá"
df_localidades["direccion"].replace({"Grandes flotas, Bogota": " Grandes flotas, Bogotá"}, inplace=True)

In [14]:
#Añadiendo según la columna localidad , su latitud, longitud en dos nuevas columnas 
geolocator = Nominatim (user_agent='maia')
df_localidades['location'] = df_localidades['direccion'].apply(geolocator.geocode)
df_localidades['latitude'] = df_localidades.location.apply(lambda loc: loc.point[0] if loc else None)
df_localidades['longitude'] = df_localidades.location.apply(lambda loc: loc.point[1] if loc else None)
#Observando el nuevo df
df_localidades

,direccion,location,latitude,longitude
0,"Calle 13, Puente Aranda, Bogotá","(Calle 13, Avenida Carrera 68, La Pradera, UPZ...",4.638626,-74.116938
1,"Suba, Suba, Bogotá","(Templo Mormón de Bogotá, UPZ El Prado, Locali...",4.708162,-74.056220
2,"Santa ana, Usaquen, Bogotá","(Santa Ana, UPZs Localidad Usaquén, Localidad ...",4.687589,-74.033244
3,"Calle 80, Engativa, Bogotá","(Calle 80, UPZ Bolivia, Siberia, Bogotá, Cota,...",4.728491,-74.126597
4,"Av Chile, Barrios unidos, Bogotá","(Avenida Chile, Calle 71A, Siete de Agosto, UP...",4.666143,-74.074637
5,"Grandes flotas, Bogotá",None,NaN,NaN


In [15]:
#Encontramos que grandes flotas puede ser por coincidencia una red de estaciones que venden un servicio, esto puede ser de nuestro interés, 
#ya que son mas de 2000, pero al dividirlo por el volumen de ventas que aportan, no tendría sentido.
#Vamos a centralizarlo en su sede a falta de nuevos datos, se debería preguntar en este caso al cliente si el punto que define es
#el origen de la flota que hace la entrega o si es la red de estaciones al final del servicio.
#Contrastar si esto fuera real, al no serlo sigo con el ejemplo del ejercicio
df_localidades["direccion"] = df_localidades["direccion"].replace({" Grandes flotas, Bogotá": "Carrera 7 75 51, Bogotá"})
df_localidades

df_localidades['location'] = df_localidades['direccion'].apply(geolocator.geocode)
df_localidades['latitude'] = df_localidades.location.apply(lambda loc: loc.point[0] if loc else None)
df_localidades['longitude'] = df_localidades.location.apply(lambda loc: loc.point[1] if loc else None)

In [17]:
df_localidades
#guardamos
df_localidades.to_csv("../data/localidades.csv")

<center><h3> Folium</h3></center>


In [18]:
#Ahora porfin podemos graficar nuestro primer mapa
market_clust = folium.Map([4.666667, -74.083333],
                        tiles='cartodbpositron',
                        size = (200,200),
                        for_ratios=2,
                        control_scale = True,
                        detect_retina=True,
                        zoom_start=12)                         
marker_cluster = MarkerCluster(name="Localidades").add_to(market_clust)
for each in df_localidades.iterrows():
        folium.Marker(location=[each[1]['latitude'], each[1]['longitude']],
            popup = each[1]['direccion'],
            icon = folium.Icon(color='black')
                    ).add_to(marker_cluster)
folium.LayerControl().add_to(market_clust)
market_clust
#lo guardamos en html 
market_clust.save('mapa1.html')

<center><h3> Plotly</h3></center>


In [19]:
#Mapeando con plotly
import plotly.express as px
fig = px.scatter_mapbox( df_localidades, lat="latitude", lon="longitude",
                    hover_name="direccion",
                    zoom=10,
                    height=300)
fig.update_layout(mapbox_style="carto-positron",
                    margin={"r":0,"t":0,"l":0,"b":0}, 
                    )
fig.show()


<div style="display:flex; flex-direction:row;">
  <div style="flex:1;">
  <center>
    <img src="http://www.idforideas.com/img/logo.png" alt="imagen1" style="background-color:white;border-radius:50%"; height="100" width="100">
        <a href="https://www.idforideas.com/" target="_blank"><p>Specials Thanks </br> IDFORIDEAS </p></a>
        </center>
  </div>
  <div style="flex:1;">
  <center>
 <img src="https://maia.run/wp-content/uploads/2023/01/Logotipo-Completo-Horizontal-MAIA-DIAPO-BLANCO-RGB-e1674195347476-768x251.png" alt="imagen1"  height="100">
        <a href="https://www.maia.run" target="_blank"><p>Mastering Artificial Intelligence Administration</p></a>
        </center>
  </div>
    <div style="flex:1;">
  <center>
 <img src="https://maia.run/wp-content/uploads/2022/12/Emblema-MAIA-DIAPO-BLANCO-RGB-150x150.png" alt="imagen1"  height="100" width="100">
        <a href="https://www.maiacorp.us" target="_blank"><p>MAIACorp, Wyoming, USA</p></a>
        </center>
</div>